In [4]:
import pandas as pd
from surprise import Dataset, Reader, CoClustering
from surprise.model_selection import GridSearchCV
from copy import deepcopy
import pickle

In [5]:
train = pd.read_csv('data/csv/train', index_col=0)
test = pd.read_csv('data/csv/test', index_col=0)
validate = pd.read_csv('data/csv/validate', index_col=0)

grid_set = pd.concat([validate, train])

In [6]:
reader = Reader(rating_scale=(0, 5))

data = Dataset.load_from_df(grid_set[['srch_id', 'prop_id', 'target']], reader)

In [12]:
%%time
param_grid = {
    "n_cltr_u": [3, 5, 7],
    "n_cltr_i": [3, 5, 7],
    "n_epochs": [10, 20, 30],
    "random_state": [42],
    "verbose": [False]
}
gs = GridSearchCV(CoClustering, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

CPU times: user 3h 10min 8s, sys: 3min 8s, total: 3h 13min 16s
Wall time: 3h 11min 44s


In [15]:
best_rmse = gs.best_estimator['rmse']
best_mae = gs.best_estimator['mae']

In [19]:
%%time
best_mae.fit(data.build_full_trainset())

<timed eval>:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


CPU times: user 1min 6s, sys: 900 ms, total: 1min 7s
Wall time: 1min 6s


In [20]:
%%time
best_rmse.fit(data.build_full_trainset())

<timed eval>:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


CPU times: user 57.2 s, sys: 714 ms, total: 57.9 s
Wall time: 57.5 s


In [25]:
gs.best_params

{'rmse': {'n_cltr_u': 3,
  'n_cltr_i': 3,
  'n_epochs': 10,
  'random_state': 42,
  'verbose': False},
 'mae': {'n_cltr_u': 7,
  'n_cltr_i': 7,
  'n_epochs': 10,
  'random_state': 42,
  'verbose': False}}

In [44]:
predictions_best_rmse = []
w = test.reset_index()
for i in w.index:
    try:
        row = w.iloc[i]
        uid = row['srch_id']
        iid = row['prop_id']   
        est = best_rmse.predict(uid, iid).est
        predictions_best_rmse.append([uid, iid, est])
        if i % 1_000_000 == 0:
            print(i)
    except:
        print(f"Index {i} failed...")

0


In [42]:
with open('model_output/Co_Clustering_rmse.csv', 'w') as out_file:
    out_file.write("SearchId,PropertyId,Estimated\n")
    for a, b, c in predictions_best_rmse:
        out_file.write(f"{a}, {b}, {c}\n")

NameError: name 'ppp' is not defined

In [56]:
best_rmse.predict('575', '6713').est

0.15633683131004508

In [45]:
predictions_best_rmse

[[25, 2863, 0.15633683131004508],
 [25, 12255, 0.15633683131004508],
 [25, 20707, 0.15633683131004508],
 [25, 22710, 0.15633683131004508],
 [25, 23228, 0.15633683131004508],
 [25, 30829, 0.15633683131004508],
 [25, 50684, 0.15633683131004508],
 [25, 53298, 0.15633683131004508],
 [25, 54036, 0.15633683131004508],
 [25, 54226, 0.15633683131004508],
 [25, 56370, 0.15633683131004508],
 [25, 61189, 0.15633683131004508],
 [25, 61885, 0.15633683131004508],
 [25, 63731, 0.15633683131004508],
 [25, 65623, 0.15633683131004508],
 [25, 66171, 0.15633683131004508],
 [25, 66209, 0.15633683131004508],
 [25, 68528, 0.15633683131004508],
 [25, 69645, 0.15633683131004508],
 [25, 79870, 0.15633683131004508],
 [25, 81772, 0.15633683131004508],
 [25, 83281, 0.15633683131004508],
 [25, 88629, 0.15633683131004508],
 [25, 91587, 0.15633683131004508],
 [25, 112716, 0.15633683131004508],
 [25, 118859, 0.15633683131004508],
 [25, 120046, 0.15633683131004508],
 [25, 120328, 0.15633683131004508],
 [25, 122279, 0.1